## Image Loading and Preprocessing:

1. Load images from dataset.
2. Preprocess images by resizing, normalizing, and handling imbalanced data to improve model performance and generalization.
3. Apply ImageDataGenerator to artificially increase the diversity of your training data 

In [41]:
# !pip install scikit-learn
# !pip list
# !pip install tflite


     ---------------------------------------- 0.0/123.6 kB ? eta -:--:--
     -------------------------------------  122.9/123.6 kB 3.6 MB/s eta 0:00:01
     -------------------------------------- 123.6/123.6 kB 2.4 MB/s eta 0:00:00


In [53]:
# from PIL import Image
# import numpy as np
# import tensorflow as tf
# import keras
# %matplotlib inline
# import matplotlib.pyplot as plt
# import sys
# sys.executable
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import scipy
# from sklearn.utils import class_weight
# from sklearn.utils import compute_class_weight
# from tensorflow.keras.applications import EfficientNetB0
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
import os


In [ ]:
# data\test
sys.executable

In [ ]:
# See class names based on the subdirectories in the training directory
class_names = sorted(os.listdir(train_path))

# Initialize label counts
label_counts = {class_name: 0 for class_name in class_names}

# Count the images per class
for images, labels in train_dataset:
    for label in labels.numpy():
        class_name = class_names[label]
        label_counts[class_name] += 1

counter = 0
# Print the class names and label counts
for class_name, count in label_counts.items():
    counter += 1
    print(counter)
    print("Class {}: {} images".format(class_name, count))


## Apply EfficientNet-Lite model



In [31]:
# load data
val_path="data/validation"
train_path="data/train"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory( 
    train_path,
    seed=2509,
    image_size=(224, 224), # Resize
    batch_size=36,
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_path,
    seed=2509,
    image_size=(224, 224), # Resize
    shuffle=False,
    batch_size=36,
)

# handle imbalanced classes with weights
labels = np.concatenate([y.numpy() for _, y in train_dataset], axis=0)
unique_labels = np.unique(labels)
updated_class_names = [label for label in unique_labels]
class_weights = compute_class_weight('balanced', classes=updated_class_names, y=labels)

def apply_class_weights(x, y):
    class_weight = tf.gather(class_weights, y)
    return x, y, class_weight

# apply class weights to each batch
augmented_train_dataset = train_dataset.map(apply_class_weights)

# Normalize validation data
def normalize_image(image, label):
    normalized_image = image / 255.0  # Normalize pixel values to [0, 1]
    return normalized_image, label

val_dataset = val_dataset.map(normalize_image)

# normalize train data + specify weights datatype
def normalize_image(image, label, value):
    normalized_image = tf.image.per_image_standardization(image)
    return normalized_image, label, value

normalized_augmented_train_dataset = augmented_train_dataset.map(normalize_image) # applying normalization to each batch

# convert labels to integer type and sample_weights to float32

def prepare_batch(x, y, w):
    return x, tf.cast(y, tf.int64), w

train_batches = normalized_augmented_train_dataset.map(prepare_batch)


Found 3115 files belonging to 36 classes.
Found 351 files belonging to 36 classes.


In [32]:
for batch_images, batch_labels, batch_weights in train_batches.take(5):
    print("Batch Labels Shape:", batch_labels.shape)
    print("Batch Weights Shape:", batch_weights.shape)
    unique_classes = np.unique(batch_labels.numpy())
    print("Unique Classes:", unique_classes)
    print("Number of Unique Classes:", len(unique_classes))


Batch Labels Shape: (36,)
Batch Weights Shape: (36,)
Unique Classes: [ 3  4  6  8  9 10 13 14 15 17 18 19 20 21 22 24 25 26 27 29 31 33 34 35]
Number of Unique Classes: 24
Batch Labels Shape: (36,)
Batch Weights Shape: (36,)
Unique Classes: [ 0  2  3  5  6  7  8  9 11 13 14 15 18 19 20 25 26 30 32 33 35]
Number of Unique Classes: 21
Batch Labels Shape: (36,)
Batch Weights Shape: (36,)
Unique Classes: [ 1  2  3  4  6  9 11 12 13 15 16 17 18 20 21 22 23 25 26 27 28 29 30 31
 32 34]
Number of Unique Classes: 26
Batch Labels Shape: (36,)
Batch Weights Shape: (36,)
Unique Classes: [ 0  1  3  5  6  9 10 12 15 16 17 18 19 21 22 24 25 26 27 28 30 31 33 35]
Number of Unique Classes: 24
Batch Labels Shape: (36,)
Batch Weights Shape: (36,)
Unique Classes: [ 2  4  6  7  9 12 13 17 18 19 20 22 24 28 31 33 34]
Number of Unique Classes: 17


In [38]:
#  building EfficientNetB0 model
input_shape = (224, 224, 3)
num_classes = 36

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output

# reshape the output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.summary()


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_12 (Rescaling)    (None, 224, 224, 3)          0         ['input_8[0][0]']             
                                                                                                  
 normalization_6 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_12[0][0]']        
 tion)                                                                                            
                                                                                                  
 rescaling_13 (Rescaling)    (None, 224, 224, 3)          0         ['normalization_6[0][0]'

In [39]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# train model
fitted_model = model.fit(train_batches, epochs=10)

Epoch 1/10
87/87 [==============================] - 1079s 12s/step - loss: 1.3536 - accuracy: 0.6189
Epoch 2/10
87/87 [==============================] - 1537s 18s/step - loss: 0.5629 - accuracy: 0.8132
Epoch 3/10
87/87 [==============================] - 1151s 13s/step - loss: 0.3709 - accuracy: 0.8709
Epoch 4/10
87/87 [==============================] - 1202s 14s/step - loss: 0.3264 - accuracy: 0.8915
Epoch 5/10
87/87 [==============================] - 1207s 14s/step - loss: 0.2802 - accuracy: 0.9165
Epoch 6/10
87/87 [==============================] - 1187s 14s/step - loss: 0.2253 - accuracy: 0.9188
Epoch 7/10
87/87 [==============================] - 1151s 13s/step - loss: 0.1843 - accuracy: 0.9396
Epoch 8/10
87/87 [==============================] - 3611s 42s/step - loss: 0.1517 - accuracy: 0.9477
Epoch 9/10
87/87 [==============================] - 1190s 14s/step - loss: 0.1943 - accuracy: 0.9387
Epoch 10/10
87/87 [==============================] - 1196s 14s/step - loss: 0.1900 - accura

In [55]:
# save model
saved_models_directory = 'Saved_Model'
os.makedirs(saved_models_directory, exist_ok=True)
model.save(os.path.join(saved_models_directory, 'my_model'))

INFO:tensorflow:Assets written to: Saved_Model\my_model\assets


INFO:tensorflow:Assets written to: Saved_Model\my_model\assets


In [57]:
path_to_saved_model = 'Saved_Model/my_model'

# Load the model
loaded_model = tf.keras.models.load_model(path_to_saved_model)

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('converted_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpbovlmcrg\assets


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpbovlmcrg\assets
